# Labour Market Statistics in Canada

In [20]:
import os
import pandas as pd
from datetime import datetime
import re
from dotenv import dotenv_values


censored_dict = dotenv_values(".env")
input_dir, output_dir = censored_dict['FILE_IMPORT_PATH'], censored_dict['FILE_EXPORT_PATH']
filename = "2020-2024_labour_force_stats.csv"
input_df = pd.read_csv(os.path.join(input_dir, filename))

In [21]:
input_df.head()

,Labour force characteristics,Data type,20-Mar,20-Apr,20-May,20-Jun,20-Jul,20-Aug,20-Sep,20-Oct,...,23-Jun,23-Jul,23-Aug,23-Sep,23-Oct,23-Nov,23-Dec,24-Jan,24-Feb,24-Mar
0,Population 6 7,"Seasonally adjusted (x 1,000)","24,530.10","24,528.70","24,526.20","24,531.20","24,536.10","24,538.80","24,542.70","24,546.80",...,"25,189.70","25,249.00","25,323.60","25,384.50","25,448.60","25,506.10","25,562.60","25,662.60","25,727.90","25,800.20"
1,NaN,Trend-cycle 8,..,..,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
2,Labour force 9,"Seasonally adjusted (x 1,000)","18,812.00","17,791.80","18,257.20","18,971.60","19,154.80","19,266.90","19,399.40","19,457.30",...,"20,257.70","20,270.30","20,282.10","20,344.80","20,385.30","20,416.20","20,435.80","20,461.80","20,553.60","20,626.30"
3,NaN,Trend-cycle 8,..,..,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
4,Employment 10,"Seasonally adjusted (x 1,000)","17,227.00","15,305.60","15,599.10","16,603.40","16,993.20","17,233.50","17,673.10","17,744.10",...,"19,135.60","19,131.00","19,153.00","19,203.40","19,216.80","19,234.30","19,256.20","19,264.90","19,332.60","19,336.70"


In [3]:
# filter out unused rows
input_df = input_df[input_df['Labour force characteristics'].notna()]

In [4]:
input_df.head()

,Labour force characteristics,Data type,20-Mar,20-Apr,20-May,20-Jun,20-Jul,20-Aug,20-Sep,20-Oct,...,23-Jun,23-Jul,23-Aug,23-Sep,23-Oct,23-Nov,23-Dec,24-Jan,24-Feb,24-Mar
0,Population 6 7,"Seasonally adjusted (x 1,000)","24,530.10","24,528.70","24,526.20","24,531.20","24,536.10","24,538.80","24,542.70","24,546.80",...,"25,189.70","25,249.00","25,323.60","25,384.50","25,448.60","25,506.10","25,562.60","25,662.60","25,727.90","25,800.20"
2,Labour force 9,"Seasonally adjusted (x 1,000)","18,812.00","17,791.80","18,257.20","18,971.60","19,154.80","19,266.90","19,399.40","19,457.30",...,"20,257.70","20,270.30","20,282.10","20,344.80","20,385.30","20,416.20","20,435.80","20,461.80","20,553.60","20,626.30"
4,Employment 10,"Seasonally adjusted (x 1,000)","17,227.00","15,305.60","15,599.10","16,603.40","16,993.20","17,233.50","17,673.10","17,744.10",...,"19,135.60","19,131.00","19,153.00","19,203.40","19,216.80","19,234.30","19,256.20","19,264.90","19,332.60","19,336.70"
6,Full-time employment 11,"Seasonally adjusted (x 1,000)","14,553.20","13,138.10","13,358.00","13,898.20","13,961.50","14,108.00","14,529.40","14,534.90",...,"15,947.90","15,920.30","15,948.50","15,964.50","15,989.90","16,028.40","16,027.80","16,029.40","16,099.50","16,110.00"
8,Part-time employment 12,"Seasonally adjusted (x 1,000)","2,673.70","2,167.40","2,241.10","2,705.20","3,031.70","3,125.50","3,143.80","3,209.30",...,"3,187.70","3,210.60","3,204.40","3,239.00","3,226.90","3,205.90","3,228.40","3,235.40","3,233.10","3,226.70"


In [5]:
def change_date_format(col_name):
    '''
        Arg: str
        Return: str
            convert YY-MMM date format to the standard YYYY-MM-DD format
            ex) 20-Feb -> 2020-02-01
            else return original col name
    '''
    if '-' in col_name:
        date_obj = datetime.strptime(col_name, "%y-%b")
        return date_obj.strftime("%Y-%m-%d")
    else:
        return col_name

# this is the dictionary that's going to be used to rename columns
input_df = input_df.drop(columns='Data type')
rename_dict = {col_name:change_date_format(col_name) for col_name in input_df.columns}
not_month_cols = {'Labour force characteristics': 'Date'}
rename_dict.update(not_month_cols)
input_df.rename(columns=rename_dict, inplace=True)

In [6]:
def remove_numbers(text):
    '''
        Arg: str
        Return: str
            remove numeric values from strings
            ex) Employment 12 -> Employment
    '''
    return re.sub(r'\d+', '', text).strip()

input_df['Date'] = [remove_numbers(category) for category in input_df['Date']]
input_df.set_index('Date', inplace=True)

In [7]:
input_df.head()

,2020-03-01,2020-04-01,2020-05-01,2020-06-01,2020-07-01,2020-08-01,2020-09-01,2020-10-01,2020-11-01,2020-12-01,...,2023-06-01,2023-07-01,2023-08-01,2023-09-01,2023-10-01,2023-11-01,2023-12-01,2024-01-01,2024-02-01,2024-03-01
Date,,,,,,,,,,,,,,,,,,,,,
Population,"24,530.10","24,528.70","24,526.20","24,531.20","24,536.10","24,538.80","24,542.70","24,546.80","24,550.50","24,547.40",...,"25,189.70","25,249.00","25,323.60","25,384.50","25,448.60","25,506.10","25,562.60","25,662.60","25,727.90","25,800.20"
Labour force,"18,812.00","17,791.80","18,257.20","18,971.60","19,154.80","19,266.90","19,399.40","19,457.30","19,402.80","19,378.20",...,"20,257.70","20,270.30","20,282.10","20,344.80","20,385.30","20,416.20","20,435.80","20,461.80","20,553.60","20,626.30"
Employment,"17,227.00","15,305.60","15,599.10","16,603.40","16,993.20","17,233.50","17,673.10","17,744.10","17,765.50","17,687.80",...,"19,135.60","19,131.00","19,153.00","19,203.40","19,216.80","19,234.30","19,256.20","19,264.90","19,332.60","19,336.70"
Full-time employment,"14,553.20","13,138.10","13,358.00","13,898.20","13,961.50","14,108.00","14,529.40","14,534.90","14,662.10","14,681.00",...,"15,947.90","15,920.30","15,948.50","15,964.50","15,989.90","16,028.40","16,027.80","16,029.40","16,099.50","16,110.00"
Part-time employment,"2,673.70","2,167.40","2,241.10","2,705.20","3,031.70","3,125.50","3,143.80","3,209.30","3,103.40","3,006.70",...,"3,187.70","3,210.60","3,204.40","3,239.00","3,226.90","3,205.90","3,228.40","3,235.40","3,233.10","3,226.70"


In [8]:
cleaned_df = input_df.transpose()

In [9]:
cleaned_df.columns.name = cleaned_df.index.name
cleaned_df.reset_index(inplace=True)
cleaned_df.rename(columns={'index':'Date'}, inplace=True)

In [10]:
cleaned_df.head()

,Date,Population,Labour force,Employment,Full-time employment,Part-time employment,Unemployment,Unemployment rate,Participation rate,Employment rate
0,2020-03-01,"24,530.10","18,812.00","17,227.00","14,553.20","2,673.70","1,585.00",8.4,76.7,70.2
1,2020-04-01,"24,528.70","17,791.80","15,305.60","13,138.10","2,167.40","2,486.20",14,72.5,62.4
2,2020-05-01,"24,526.20","18,257.20","15,599.10","13,358.00","2,241.10","2,658.10",14.6,74.4,63.6
3,2020-06-01,"24,531.20","18,971.60","16,603.40","13,898.20","2,705.20","2,368.20",12.5,77.3,67.7
4,2020-07-01,"24,536.10","19,154.80","16,993.20","13,961.50","3,031.70","2,161.60",11.3,78.1,69.3


In [11]:
cleaned_df.to_csv(os.path.join(output_dir, "2020-2024_cleaned_labour_market_data.csv"), index=False)

In [44]:
import psycopg2


DB_NAME = censored_dict['DB_NAME']
DB_USER = censored_dict['DB_USER']
DB_PASS = censored_dict['DB_PASS']
DB_HOST = censored_dict['DB_HOST']
DB_PORT = censored_dict['DB_PORT']

conn = psycopg2.connect(
    dbname=DB_NAME,
    user=DB_USER,
    password=DB_PASS,
    host=DB_HOST,
    port=DB_PORT,
)

cur = conn.cursor()

In [40]:
cur.execute("select * from labour_tables.labour_stats")

rows = cur.fetchall()
df = pd.DataFrame(rows, columns=[desc[0] for desc in cur.description])
# Close the cursor and connection
cur.close()
conn.close()

In [43]:
df.head(2)

,date,population,labour_force,employment,full_time,part_time,unemployed,unemployment_rate,participation_rate,employment_rate
0,2020-03-01,24530.10,18812.00,17227.00,14553.20,2673.70,1585.00,8.4,76.7,70.2
1,2020-04-01,24528.70,17791.80,15305.60,13138.10,2167.40,2486.20,14,72.5,62.4


In [ ]:
cur.execute(
    """
	CREATE TABLE labour_tables.labour_stats2 (
		Date DATE NOT NULL,
		Population NUMERIC,
		Labour_force NUMERIC,
		Employment NUMERIC,
		Full_time NUMERIC,
		Part_time NUMERIC,
		Unemployed NUMERIC,
		Unemployment_rate NUMERIC,
		Participation_rate NUMERIC,
		Employment_rate NUMERIC);
    """
)
